# Variant Calling Analysis

## Required packages

In [1]:
# Python 3.9 or higher (.removesuffix() in  SAMtools realign NCBI RefSeq assembly)
from tkinter import filedialog
import os
import ipywidgets as widgets
from IPython.display import display
from datetime import datetime
import json
import time
import shutil
import requests
import zipfile
import subprocess
import gzip
from collections import defaultdict
import pandas as pd
import csv

## Directory Structure

In [2]:
cwd = os.getcwd()

main_directory_path = filedialog.askdirectory(title="Project", initialdir=cwd)
#main_directory_path = "/home/student/Variant_Calling/Jupyter/PAO1"
Fastq_directory_path = filedialog.askdirectory(title="Fastq files", initialdir=cwd)
#Fastq_directory_path = os.path.join(main_directory_path, "Fastq")

main_subdirectories = ["Analysis"]

for directory in  main_subdirectories:
    main_subdirectory_path = os.path.join(main_directory_path, directory)
    if not os.path.exists(main_subdirectory_path):
        os.mkdir(main_subdirectory_path)

analysis_directory_path = os.path.join(main_directory_path, "Analysis")
analysis_subdirectories = ["fastq","assemblies","variant_calling","samfiles", "genomes"]

for directory in analysis_subdirectories:
    analysis_subdirectory_path = os.path.join(analysis_directory_path, directory)
    if not os.path.exists(analysis_subdirectory_path):
        os.mkdir(analysis_subdirectory_path)

genomes_directory_path = os.path.join(analysis_directory_path, "genomes")
genomes_subdirectories = ["annotated", "indexed"]

for directory in genomes_subdirectories:
    genomes_subdirectory_path = os.path.join(genomes_directory_path, directory)
    if not os.path.exists(genomes_subdirectory_path):
        os.mkdir(genomes_subdirectory_path)

variant_calling_directory_path = os.path.join(analysis_directory_path, "variant_calling")
variant_calling_subdirectories = ["annotated","curated","indels","intermediate","raw","snps","totalpileup","vcf","results"]

for directory in variant_calling_subdirectories:
    vc_subdirectory_path = os.path.join(variant_calling_directory_path, directory)
    if not os.path.exists(vc_subdirectory_path):
        os.mkdir(vc_subdirectory_path)

# Analysis subdirectories path
fastq_directory_path = os.path.join(analysis_directory_path, "fastq")
samfiles_directory_path = os.path.join(analysis_directory_path, "samfiles")
assemblies_directory_path = os.path.join(analysis_directory_path, "assemblies")

## Variant Calling subdirectories path
totalpileup_directory_path = os.path.join(variant_calling_directory_path, "totalpileup")
raw_directory_path = os.path.join(variant_calling_directory_path, "raw")
intermediate_directory_path = os.path.join(variant_calling_directory_path, "intermediate")
vcf_directory_path = os.path.join(variant_calling_directory_path, "vcf")
indels_directory_path = os.path.join(variant_calling_directory_path, "indels")
snps_directory_path = os.path.join(variant_calling_directory_path, "snps")
curated_directory_path = os.path.join(variant_calling_directory_path, "curated")
annotated_directory_path = os.path.join(variant_calling_directory_path, "annotated")
results_directory_path = os.path.join(variant_calling_directory_path, "results")

## Genomes subdirectories path
indexed_directory_path = os.path.join(genomes_directory_path, "indexed")
annotated_genomes_directory_path = os.path.join(genomes_directory_path, "annotated")

# Softwares
picard_jar = "/home/student/miniconda3/envs/VariantCalling/share/picard-3.4.0-0/picard.jar"
samtools = "/home/student/miniconda3/envs/VariantCalling/bin/samtools"
bowtie2_build = "/home/student/miniconda3/envs/VariantCalling/bin/bowtie2-build"
bowtie2 = "/home/student/miniconda3/envs/VariantCalling/bin/bowtie2"
GATK_jar = "/home/student/Programs/GenomeAnalysisTK-3.4-46/GenomeAnalysisTK.jar"
java_8 = "/home/student/javav/jdk1.8.0_131/bin/java"
java_21 = "/home/student/javav/jdk-21.0.6/bin/java"
snpEff_jar = "/home/student/miniconda3/envs/VariantCalling/share/snpeff-5.2-1/snpEff.jar"
perl = "/home/student/miniconda3/envs/VariantCalling/bin/perl"
convert2annovar = "/home/student/annovar/convert2annovar.pl"
vcfEffOnePerLine = "/home/student/miniconda3/envs/VariantCalling/share/snpeff-5.2-1/scripts/vcfEffOnePerLine.pl"
snpSift_jar = "/home/student/miniconda3/envs/VariantCalling/share/snpsift-5.2-0/SnpSift.jar"
spades = "/home/student/miniconda3/envs/VariantCalling/bin/spades.py"
buildDbNcbi = "/home/student/miniconda3/envs/VariantCalling/share/snpeff-5.2-1/scripts/buildDbNcbi.sh"
snpeff_dir_path = "/home/student/miniconda3/envs/VariantCalling/share/snpeff-5.2-1/"

## Configuration

In [3]:
# Load configuration file widget
configuration_widget = widgets.Checkbox(description='Configuration file', value=False)

configuration_inp_box = widgets.VBox()
configuration_section = widgets.VBox([configuration_widget, configuration_inp_box])

def update_configuration_inputs(num):
    items = []
    for i in range(num):
        upload_file = widgets.FileUpload(accept='.json', multiple=False,
                                        layout=widgets.Layout(width='300px'))
        items.append(upload_file)
    configuration_inp_box.children = items
    configuration_inp_box.layout.display = 'block' if num > 0 else 'none'

def on_configuration_widget_change(change):
    if change['name'] == 'value':
        update_configuration_inputs(change['new'])

configuration_widget.observe(on_configuration_widget_change, names='value')
update_configuration_inputs(configuration_widget.value)

# Merge widget
merge_widget = widgets.Checkbox(description='Merge .fastq files', value=False)

# Download reference genome/s widget
download_widget = widgets.BoundedIntText(description='Dowload reference genome', value=0,
                                         min=0, max=10, step=1,
                                        style={'description_width':'200px'})

download_inp_box = widgets.VBox()
download_section = widgets.VBox([download_widget, download_inp_box])

def update_download_inputs(num):
    items = []
    for i in range(num):
        textbox = widgets.Text(description=f'Assembly Accession {i+1}',
                               placeholder='GCF_000006765.1',
                               style={'description_width':'150px'},
                               layout=widgets.Layout(width='300px'))
        items.append(textbox)
    download_inp_box.children = items
    download_inp_box.layout.display = 'block' if num > 0 else 'none'

def on_download_widget_change(change):
    if change['name'] == 'value':
        update_download_inputs(change['new'])

download_widget.observe(on_download_widget_change, names='value')
update_download_inputs(download_widget.value)

# Genome indexing widget
index_widget = widgets.BoundedIntText(description='Genome indexing', value=0,
                                     min=0, max=10, step=1,
                                     style={'description_width':'150px'})

index_inp_box = widgets.VBox()
index_section = widgets.VBox([index_widget, index_inp_box])

def update_index_inputs(num):
    items = []
    for i in range(num):
        textbox = widgets.Text(description=f'Assembly Accession {i+1}',
                               placeholder='GCF_000006765.1',
                               style={'description_width':'150px'},
                               layout=widgets.Layout(width='300px'))
        items.append(textbox)
    index_inp_box.children = items
    index_inp_box.layout.display = 'block' if num > 0 else 'none'

def on_index_widget_change(change):
    if change['name'] == 'value':
        update_index_inputs(change['new'])

index_widget.observe(on_index_widget_change, names='value')
update_index_inputs(index_widget.value)

# Bowtie2 widget
bowtie2_widget = widgets.Checkbox(description='Bowtie2 alignment', value=False)

alignment_widget = widgets.BoundedIntText(description='Strains', value=1,
                                         min=1, max=10, step=1,
                                         style={'description_width':'150px'})

alignment_inp_box = widgets.VBox()
alignment_section = widgets.VBox([alignment_widget, alignment_inp_box])

def update_alignment_inputs(num):
    items = []
    for i in range(num):
        textbox = widgets.Text(description=f'Assembly Accession {i+1}',
                              placeholder='GCF_000006765.1',
                              style={'description_width':'150px'},
                              layout=widgets.Layout(width='300px'))
        file_upload = widgets.FileUpload(accept='.txt', multiple=False,
                                        layout=widgets.Layout(width='300px'))
        input_pair = widgets.HBox([textbox, file_upload])
        items.append(input_pair)
    alignment_inp_box.children = items

def on_alignment_widget_change(change):
    if change['name'] == 'value':
        update_alignment_inputs(change['new'])

alignment_widget.observe(on_alignment_widget_change, names='value')
update_alignment_inputs(alignment_widget.value)

def toggle_alignment_section(change):
    alignment_section.layout.display = 'block' if change['new'] else 'none'

bowtie2_widget.observe(toggle_alignment_section, names='value')
alignment_section.layout.display = 'block' if bowtie2_widget.value else 'none'

# SAMtools realign assembly widget
realign_widget = widgets.Checkbox(description='SAMtools realign assembly', value=False)

realignment_widget = widgets.BoundedIntText(description='Strains', value=1,
                                         min=1, max=10, step=1,
                                         style={'description_width':'150px'})

realignment_inp_box = widgets.VBox()
realignment_section = widgets.VBox([realignment_widget, realignment_inp_box])

def update_realignment_inputs(num):
    items = []
    for i in range(num):
        textbox = widgets.Text(description=f'Assembly Accession {i+1}',
                              placeholder='GCF_000006765.1',
                              style={'description_width':'150px'},
                              layout=widgets.Layout(width='300px'))
        file_upload = widgets.FileUpload(accept='.txt', multiple=False,
                                        layout=widgets.Layout(width='300px'))
        input_pair = widgets.HBox([textbox, file_upload])
        items.append(input_pair)
    realignment_inp_box.children = items

def on_realignment_widget_change(change):
    if change['name'] == 'value':
        update_realignment_inputs(change['new'])

realignment_widget.observe(on_realignment_widget_change, names='value')
update_realignment_inputs(realignment_widget.value)

def toggle_realignment_section(change):
    realignment_section.layout.display = 'block' if change['new'] else 'none'

realign_widget.observe(toggle_realignment_section, names='value')
realignment_section.layout.display = 'block' if realign_widget.value else 'none'

# Find SNPs
snps_widget = widgets.Checkbox(description='Find SNPs', value=False)

# Find indels
indels_widget = widgets.Checkbox(description='Find indels', value=False)

# Build genome database (snpEff) widget
build_widget = widgets.Checkbox(description='Build genome database', value=False)

database_widget = widgets.BoundedIntText(description='Databases', value=1,
                                         min=1, max=10, step=1,
                                         style={'description_width':'150px'})

database_inp_box = widgets.VBox()
database_section = widgets.VBox([database_widget, database_inp_box])

def update_database_inputs(num):
    items = []
    for i in range(num):
        textbox = widgets.Text(description=f'NCBI RefSeq {i+1}',
                               placeholder='NC_002516.2',
                               style={'description_width':'150px'},
                               layout=widgets.Layout(width='300px'))
        items.append(textbox)
    database_inp_box.children = items
    database_inp_box.layout.display = 'block' if build_widget.value else 'none'

def on_database_widget_change(change):
    if change['name'] == 'value':
        update_database_inputs(change['new'])

database_widget.observe(on_database_widget_change, names='value')
update_database_inputs(database_widget.value)

def toggle_database_section(change):
    database_section.layout.display = 'block' if change['new'] else 'none'
    if change['new']:
        update_database_inputs(database_widget.value)

build_widget.observe(toggle_database_section, names='value')
database_section.layout.display = 'block' if build_widget.value else 'none'

# snpEff annotation
snpeff_widget = widgets.Checkbox(description='SNP/indel annotation', value=False)

ncbi_refseq_widget = widgets.BoundedIntText(description='NCBI RefSeq', value=1,
                                         min=1, max=10, step=1,
                                         style={'description_width':'150px'})

ncbi_inp_box = widgets.VBox()
ncbi_section = widgets.VBox([ncbi_refseq_widget, ncbi_inp_box])

def update_ncbi_inputs(num):
    items = []
    for i in range(num):
        textbox = widgets.Text(description=f'NCBI RefSeq {i+1}',
                              placeholder='NC_002516.2',
                              style={'description_width':'150px'},
                              layout=widgets.Layout(width='300px'))
        file_upload = widgets.FileUpload(accept='.txt', multiple=False,
                                        layout=widgets.Layout(width='300px'))
        input_pair = widgets.HBox([textbox, file_upload])
        items.append(input_pair)
    ncbi_inp_box.children = items

def on_ncbi_refseq_widget_change(change):
    if change['name'] == 'value':
        update_ncbi_inputs(change['new'])

ncbi_refseq_widget.observe(on_ncbi_refseq_widget_change, names='value')
update_ncbi_inputs(ncbi_refseq_widget.value)

def toggle_ncbi_section(change):
    ncbi_section.layout.display = 'block' if change['new'] else 'none'

snpeff_widget.observe(toggle_ncbi_section, names='value')
ncbi_section.layout.display = 'block' if snpeff_widget.value else 'none'

# Filter SNP/indel by parental
filter_widget = widgets.Checkbox(description='Filter SNP/indel by parental', value=False)

parental_widget = widgets.BoundedIntText(description='Parental', value=1,
                                         min=1, max=10, step=1,
                                         style={'description_width':'150px'})

parental_inp_box = widgets.VBox()
parental_section = widgets.VBox([parental_widget, parental_inp_box])

def update_parental_inputs(num):
    items = []
    for i in range(num):
        textbox = widgets.Text(description=f'Parental {i+1}',
                              placeholder='PAO1',
                              style={'description_width':'150px'},
                              layout=widgets.Layout(width='300px'))
        file_upload = widgets.FileUpload(accept='.txt', multiple=False,
                                        layout=widgets.Layout(width='300px'))
        input_pair = widgets.HBox([textbox, file_upload])
        items.append(input_pair)
    parental_inp_box.children = items

def on_parental_widget_change(change):
    if change['name'] == 'value':
        update_parental_inputs(change['new'])

parental_widget.observe(on_parental_widget_change, names='value')
update_parental_inputs(parental_widget.value)

def toggle_parental_section(change):
    parental_section.layout.display = 'block' if change['new'] else 'none'

filter_widget.observe(toggle_parental_section, names='value')
parental_section.layout.display = 'block' if filter_widget.value else 'none'

# Create sample/mutation table widget
table_widget = widgets.Checkbox(description='Sample/mutation table', value=False)

threshold_widget = widgets.BoundedIntText(description='Threshold', value=1,
                                         min=1, step=1,
                                         style={'description_width':'150px'})

strains_widget = widgets.BoundedIntText(description='Strains', value=1,
                                        min=1, max=10, step=1,
                                        style={'description_width':'150px'})

strains_inp_box = widgets.VBox()
strains_section = widgets.VBox([strains_widget, strains_inp_box])

def update_strains_inputs(num):
    items = []
    for i in range(num):
        textbox = widgets.Text(description=f'Strain {i+1}',
                              placeholder='PAO1',
                              style={'description_width':'150px'},
                              layout=widgets.Layout(width='300px'))
        file_upload = widgets.FileUpload(accept='.txt', multiple=False,
                                        layout=widgets.Layout(width='300px'))
        input_pair = widgets.HBox([textbox, file_upload])
        items.append(input_pair)
    strains_inp_box.children = items

def on_strains_widget_change(change):
    if change['name'] == 'value':
        update_strains_inputs(change['new'])

strains_widget.observe(on_strains_widget_change, names='value')
update_strains_inputs(strains_widget.value)

table_children_box = widgets.VBox([threshold_widget, strains_section])
table_section = widgets.VBox([table_widget, table_children_box])
table_children_box.layout.display = 'block' if table_widget.value else 'none'

def toggle_table_section(change):
    table_children_box.layout.display = 'block' if change['new'] else 'none'

table_widget.observe(toggle_table_section, names='value')

# De novo genome assembly (SPAdes) widget
denovo_widget = widgets.Checkbox(description='De novo genome assembly', value=False)

# Compress large files widget
compress_widget = widgets.Checkbox(description='Compress large files', value=False,
                           layout=widgets.Layout(width='300px'))

# Comfirm selected options widget
confirm_options = widgets.Checkbox(description='Confirm', value=False, indent=False)

# configuration_widget behavior
def set_disabled_recursive_configuration(widget, disabled=True):
    if isinstance(widget, widgets.Box):
        new_children = []
        for child in widget.children:
            if isinstance(child, widgets.FileUpload):
                # Create a new FileUpload widget
                new_widget = widgets.FileUpload(
                    accept=child.accept,
                    multiple=child.multiple,
                    layout=child.layout
                )
                new_children.append(new_widget)
            else:
                set_disabled_recursive_configuration(child, disabled)
                new_children.append(child)
        widget.children = tuple(new_children)
    else:
        if hasattr(widget, 'disabled'):
            widget.disabled = disabled
        if isinstance(widget, widgets.Checkbox):
            widget.value = False
        elif isinstance(widget, widgets.BoundedIntText):
            widget.value = 0

# confirm_widget behavior
def set_disabled_recursive_confirm(widget, disabled=True):
    widget.disabled = disabled
    if hasattr(widget, 'children'):
        for child in widget.children:
            set_disabled_recursive_confirm(child, disabled)

def on_configuration_widget(change):
    if change['name'] == 'value':
        update_configuration_inputs(change['new'])
        state = change['new']

        merge_widget.disabled = state
        merge_widget.value = False

        download_widget.disabled = state
        download_widget.value = False
        set_disabled_recursive_configuration(download_section, state)

        index_widget.disabled = state
        index_widget.value = False
        set_disabled_recursive_configuration(index_section, state)
        
        bowtie2_widget.disabled = state
        bowtie2_widget.value = False
        set_disabled_recursive_configuration(alignment_section, state)
        
        realign_widget.disabled = state
        realign_widget.value = False
        set_disabled_recursive_configuration(realignment_section, state)
        
        snps_widget.disabled = state
        snps_widget.value = False
        
        indels_widget.disabled = state
        indels_widget.value = False
        
        build_widget.disabled = state
        build_widget.value = False
        set_disabled_recursive_configuration(database_section, state)
        
        snpeff_widget.disabled = state
        snpeff_widget.value = False
        set_disabled_recursive_configuration(ncbi_section, state)
        
        filter_widget.disabled = state
        filter_widget.value = False
        set_disabled_recursive_configuration(parental_section, state)

        table_widget.disabled = state
        table_widget.value = False
        set_disabled_recursive_configuration(strains_section, state)
        
        denovo_widget.disabled = state
        denovo_widget.value = False

        compress_widget.disabled = state
        compress_widget.value = False

def on_confirm_options_widget(change):
    if change['type'] == 'change' and change['name'] == 'value':
        state = change['new']

        set_disabled_recursive_confirm(configuration_section, state)
        merge_widget.disabled = state
        set_disabled_recursive_confirm(download_section, state)
        set_disabled_recursive_confirm(index_section, state)
        set_disabled_recursive_confirm(bowtie2_widget, state)
        set_disabled_recursive_confirm(alignment_section, state)
        set_disabled_recursive_confirm(realign_widget, state)
        set_disabled_recursive_confirm(realignment_section, state)
        snps_widget.disabled = state
        indels_widget.disabled = state
        set_disabled_recursive_confirm(build_widget, state)
        set_disabled_recursive_confirm(database_section, state)
        set_disabled_recursive_confirm(snpeff_widget, state)
        set_disabled_recursive_confirm(ncbi_section, state)
        set_disabled_recursive_confirm(filter_widget, state)
        set_disabled_recursive_confirm(parental_section, state)
        set_disabled_recursive_confirm(table_section, state)
        denovo_widget.disabled = state
        compress_widget.disabled = state

configuration_widget.observe(on_configuration_widget, names='value')
confirm_options.observe(on_confirm_options_widget, names='value')

column = widgets.VBox([
    configuration_section,
    merge_widget,
    download_section,
    index_widget,
    index_inp_box,
    bowtie2_widget,
    alignment_section,
    realign_widget,
    realignment_section,
    snps_widget,
    indels_widget,
    build_widget,
    database_section,
    snpeff_widget,
    ncbi_section,
    filter_widget,
    parental_section,
    table_section,
    denovo_widget,
    compress_widget,
    confirm_options
])

display(column)

### Configuration file

In [4]:
if configuration_widget.value:
    # Load configuration from json file
    try:
        configuration_file = configuration_inp_box.children[0].value[0]["content"]
        configuration = json.loads(bytes(configuration_file).decode("utf-8"))
        print("Configuration file uploaded successfully.")
        
    except IndexError:
        print("No file uploaded.")
    except KeyError:
        print("Failed to load configuration file.")
    
else:
    # Load parental SNP/indel files
    if filter_widget.value:
        parental_files = []
        for i in range(0, parental_widget.value):
            strain = parental_inp_box.children[i].children[0].value
            snp_file = filedialog.askopenfile(title=f"{strain} SNPs", initialdir="/home/student/Variant_Calling/Jupyter")
            indels_file = filedialog.askopenfile(title=f"{strain} indels", initialdir="/home/student/Variant_Calling/Jupyter")

            parental_files.append((snp_file, indels_file))

    # Create configuration
    configuration = {
        "merge": merge_widget.value,
        "download": {
            "n": download_widget.value,
            "assemblies": [box.value for box in download_inp_box.children]
        },
        "indexing": {
            "n": index_widget.value,
            "assemblies": [box.value for box in index_inp_box.children]
        },
        "bowtie2": {
            "enabled": bowtie2_widget.value,
            "n": alignment_widget.value,
            "organisms": [
                {
                    "accession": pair.children[0].value,
                    "file": pair.children[1].value[0]['name']
                        if pair.children[1].value and len(pair.children[1].value) > 0
                        else None,
                    "samples": [
                        line.strip()
                        for line in bytes(pair.children[1].value[0]['content']).decode("utf-8").splitlines()
                        if line.strip()
                    ] if pair.children[1].value and len(pair.children[1].value) > 0 else []
            }
            for pair in alignment_inp_box.children
        ]
    },
        "realign": {
            "enabled": realign_widget.value,
            "n": realignment_widget.value,
            "organisms": [
                {
                    "accession": pair.children[0].value,
                    "file": pair.children[1].value[0]['name']
                    if pair.children[1].value and len(pair.children[1].value) > 0
                    else None,
                    "samples": [
                        line.strip()
                        for line in bytes(pair.children[1].value[0]['content']).decode("utf-8").splitlines()
                        if line.strip()
                    ] if pair.children[1].value and len(pair.children[1].value) > 0 else []
                }
                for pair in realignment_inp_box.children
            ]
        },
        "snps": snps_widget.value,
        "indels": indels_widget.value,
        "build": {
            "enabled": build_widget.value,
            "n": database_widget.value,
            "databases": [box.value for box in database_inp_box.children]
        },
        "SNP_indel_annotation": {
            "enabled": snpeff_widget.value,
            "n": ncbi_refseq_widget.value,
            "Reference_Sequences": [
                {
                    "accession": pair.children[0].value,
                    "file": pair.children[1].value[0]['name']
                        if pair.children[1].value and len(pair.children[1].value) > 0
                        else None,
                    "samples": [
                        line.strip()
                        for line in bytes(pair.children[1].value[0]['content']).decode("utf-8").splitlines()
                        if line.strip()
                    ] if pair.children[1].value and len(pair.children[1].value) > 0 else []
            }
            for pair in ncbi_inp_box.children
        ]
    },
        "filter_by_parental": {
            "enabled": filter_widget.value,
            "n": parental_widget.value,
            "Parental": [
                {
                    "strain": pair.children[0].value,
                    "snps_path": snp.name,
                    "indels_path": indels.name,
                    "file": pair.children[1].value[0]['name']
                        if pair.children[1].value and len(pair.children[1].value) > 0
                        else None,
                    "samples": [
                        line.strip()
                        for line in bytes(pair.children[1].value[0]['content']).decode("utf-8").splitlines()
                        if line.strip()
                    ] if pair.children[1].value and len(pair.children[1].value) > 0 else []
            }
            for pair, (snp, indels) in zip(parental_inp_box.children, parental_files)
        ]
    },
        "table": {
            "enabled": table_widget.value,
            "threshold": threshold_widget.value,
            "n": strains_widget.value,
            "strains": [
                {
                    "strain": pair.children[0].value,
                    "file": pair.children[1].value[0]['name']
                        if pair.children[1].value and len(pair.children[1].value) > 0
                        else None,
                    "samples": [
                        line.strip()
                        for line in bytes(pair.children[1].value[0]['content']).decode("utf-8").splitlines()
                        if line.strip()
                    ] if pair.children[1].value and len(pair.children[1].value) > 0 else []
                }
                for pair in strains_inp_box.children
            ]
        },
        "denovo": denovo_widget.value,
        "compress": compress_widget.value
    }

    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    configuration["generated_at"] = now

    config_file_path = os.path.join(main_directory_path, "config.json")

    # Create configuration file
    with open(config_file_path, 'w') as config_file:
        json.dump(configuration, config_file, indent=4)

    print("Configuration file create in", config_file_path)

# Initialize sample dictionaries
bowtie2_sample_genome_dict = {}
realign_sample_genome_dict = {}
annotation_sample_genome_dict = {}
filter_sample_strain_dict = {}
table_sample_strain_dict = {}

for organism in configuration["bowtie2"]["organisms"]:
    accession = organism["accession"]
    samples = organism["samples"]
    for sample in samples:
        if sample not in bowtie2_sample_genome_dict:
            bowtie2_sample_genome_dict[sample] = accession
        else:
            print(f"{sample} is duplicated.")

for organism in configuration["realign"]["organisms"]:
    accession = organism["accession"]
    samples = organism["samples"]
    for sample in samples:
        if sample not in realign_sample_genome_dict:
            realign_sample_genome_dict[sample] = accession
        else:
            print(f"{sample} is duplicated.")

for refseq in configuration["SNP_indel_annotation"]["Reference_Sequences"]:
    accession = refseq["accession"]
    samples = refseq["samples"]
    for sample in samples:
        if sample not in annotation_sample_genome_dict:
            annotation_sample_genome_dict[sample] = accession
        else:
            print(f"{sample} is duplicated.")

for strain in configuration["filter_by_parental"]["Parental"]:
    parental = strain["strain"]
    samples = strain["samples"]
    for sample in samples:
        if sample not in filter_sample_strain_dict:
            filter_sample_strain_dict[sample] = parental
        else:
            print(f"{sample} is duplicated.")

for strain in configuration["table"]["strains"]:
    strain_name = strain["strain"]
    for sample in strain["samples"]:
        if sample not in table_sample_strain_dict:
            table_sample_strain_dict[sample] = strain_name
        else:
            print(f"{sample} is duplicated.")

Configuration file uploaded successfully.


## Merge fastq files

In [5]:
# Merge fastq files
if configuration["merge"]:

    files = os.listdir(Fastq_directory_path)

    samples = {file.split("_")[0] for file in files}

    print("These samples are going to be processed: ")
    for sample in samples:
        print(f"  - {sample}")
    
    print()

    for sample in samples:
        start_time = time.time()
    
        L1_directory = [file for file in files if file.startswith(f"{sample}_L1")]
        L1_directory_path = os.path.join(Fastq_directory_path, "".join(L1_directory))

        L2_directory = [file for file in files if file.startswith(f"{sample}_L2")]
        L2_directory_path = os.path.join(Fastq_directory_path, "".join(L2_directory))

        L1_files = os.listdir(L1_directory_path)
        L2_files = os.listdir(L2_directory_path)
    
        L1_R1_file = [file for file in L1_files if "_R1_" in file]
        L1_R1_file_path = os.path.join(L1_directory_path, "".join(L1_R1_file))
        L2_R1_file = [file for file in L2_files if "_R1_" in file]
        L2_R1_file_path = os.path.join(L2_directory_path, "".join(L2_R1_file))

        R1_files_path = [L1_R1_file_path, L2_R1_file_path]

        L1_R2_file = [file for file in L1_files if "_R2_" in file]
        L1_R2_file_path = os.path.join(L1_directory_path, "".join(L1_R2_file))
        L2_R2_file = [file for file in L2_files if "_R2_" in file]
        L2_R2_file_path = os.path.join(L2_directory_path, "".join(L2_R2_file))

        R2_files_path = [L1_R2_file_path, L2_R2_file_path]

        ## Merge R1 files
        merged_R1_file = f"{sample}_R1.fastq.gz"
        merged_R1_file_path = os.path.join(fastq_directory_path, merged_R1_file)

        with open(merged_R1_file_path, 'wb') as R1_file:
            for file in R1_files_path:
                with open(file, 'rb') as my_file:
                    shutil.copyfileobj(my_file, R1_file)

        ## Merge R2 files
        merged_R2_file = f"{sample}_R2.fastq.gz"
        merged_R2_file_path = os.path.join(fastq_directory_path, merged_R2_file)

        with open(merged_R2_file_path, 'wb') as R2_file:
            for file in R2_files_path:
                with open(file, 'rb') as my_file:
                    shutil.copyfileobj(my_file, R2_file)
    
        end_time = time.time()

        execution_time = end_time - start_time
        print(f"{sample} processed in {execution_time:.2f} seconds.")

    print()
    print("Merge finished!")

These samples are going to be processed: 
  - PAO1wt12P
  - PA14wt0253G
  - PA14wt0251G

PAO1wt12P processed in 9.74 seconds.
PA14wt0253G processed in 20.59 seconds.
PA14wt0251G processed in 8.75 seconds.

Merge finished!


## Download reference genomes

In [6]:
if configuration["download"]["n"]:

    assemblies = configuration["download"]["assemblies"]

    # Download reference genome .fastq file from NCBI
    for assembly_id in assemblies:
        url = f"https://api.ncbi.nlm.nih.gov/datasets/v2/genome/accession/{assembly_id}/download?include_annotation_type=GENOME_FASTA&hydrated=FULLY_HYDRATED"

        try:
            response = requests.get(url, stream=True, timeout=60)
            response.raise_for_status()
        except requests.exceptions.RequestException as download_error:
            print(f"Error downloading the assembly {assembly_id}: {download_error}")
            continue

        if response.status_code == 200:
            ref_genome_file = f"{assembly_id}.zip"
            ref_genome_file_path = os.path.join(genomes_directory_path, ref_genome_file)
            with open(ref_genome_file_path, 'wb') as genome_file:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        genome_file.write(chunk)
            print(f"NCBI RefSeq assembly {assembly_id} downloaded successfully.")

            assembly_id_directory = os.path.join(genomes_directory_path, assembly_id)
            if not os.path.exists(assembly_id_directory):
                os.mkdir(assembly_id_directory)
            
            with zipfile.ZipFile(ref_genome_file_path, 'r') as zip_ref:
                zip_ref.extractall(path=assembly_id_directory)
            print(f"NCBI RefSeq assembly {assembly_id} unzipped successfully.")
            print()

            os.remove(ref_genome_file_path)
        
        else:
            print(f"Error downloading the NCBI RefSeq assembly {assembly_id}")

NCBI RefSeq assembly GCF_000006765.1 downloaded successfully.
NCBI RefSeq assembly GCF_000006765.1 unzipped successfully.

NCBI RefSeq assembly GCF_000014625.1 downloaded successfully.
NCBI RefSeq assembly GCF_000014625.1 unzipped successfully.



## Genome indexing

In [7]:
if configuration["indexing"]["n"]:

    assemblies = configuration["indexing"]["assemblies"]

    for assembly_id in assemblies:
        print(f"Indexing {assembly_id} genome")
        assembly_fasta_file_directory_path = os.path.join(genomes_directory_path, assembly_id,
                                                          "ncbi_dataset/data", assembly_id)
    
        fasta_file = os.listdir(assembly_fasta_file_directory_path)
        fasta_file_path = os.path.join(assembly_fasta_file_directory_path, "".join(fasta_file))

        indexed_assembly_directory_path = os.path.join(genomes_directory_path, "indexed", assembly_id)
        if not os.path.exists(indexed_assembly_directory_path):
            os.mkdir(indexed_assembly_directory_path)
        
        N_fasta_file_path = os.path.join(indexed_assembly_directory_path, f"N_{assembly_id}.fasta")

        # Normalize Fasta
        command = [
            "java", "-jar", picard_jar, "NormalizeFasta",
            "I=" + fasta_file_path,
            "O=" + N_fasta_file_path
        ]

        subprocess.run(command, check=True)

        # Creating .dict file
        reference = N_fasta_file_path
        dict_file_path = os.path.join(indexed_assembly_directory_path, f"N_{assembly_id}.dict")

        if os.path.exists(dict_file_path):
            print(f"{assembly_id}.dict already exists.")
        else:
            command = [
                "java", "-jar", picard_jar, "CreateSequenceDictionary",
                "R=" + reference,
                "O=" + dict_file_path
            ]

            subprocess.run(command, check=True)

        # Creating .fasta.fai files
        command = [
            samtools, "faidx", reference
        ]

        subprocess.run(command, check=True)

        # Building bowtie2 index
        index_directory_path = os.path.join(indexed_assembly_directory_path, "index")

        if not os.path.exists(index_directory_path):
            os.mkdir(index_directory_path)

        index_file_path = os.path.join(index_directory_path, f"{assembly_id}_index")
    
        command = [
            bowtie2_build, reference, index_file_path
        ]

        subprocess.run(command, check=True)
        print(f"{assembly_id} indexed successfully!")

Indexing GCF_000006765.1 genome


INFO	2025-06-04 09:04:51	NormalizeFasta	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** 
https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    NormalizeFasta -I /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/GCF_000006765.1/ncbi_dataset/data/GCF_000006765.1/GCF_000006765.1_ASM676v1_genomic.fna -O /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000006765.1/N_GCF_000006765.1.fasta
**********


09:04:51.613 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/student/miniconda3/envs/VariantCalling/share/picard-3.4.0-0/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Jun 04 09:04:51 CEST 2025] NormalizeFasta INPUT=/media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/GCF_000006765.1/n

GCF_000006765.1.dict already exists.
Settings:
  Output files: "/media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000006765.1/index/GCF_000006765.1_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000006765.1/N_GCF_000006765.1.fasta
Reading reference sizes


Building a SMALL index


  Time reading reference sizes: 00:00:01
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 1566101
Using parameters --bmax 1174576 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 1174576 --dcv 1024
Constructing suffix-array element generator
Building DifferenceCoverSample
  Building sPrime
  Building sPrimeOrder
  V-Sorting samples
  V-Sorting samples time: 00:00:00
  Allocating rank array
  Ranking v-sort output
  Ranking v-sort output time: 00:00:00
  Invoking Larsson-Sadakane on ranks
  Invoking Larsson-Sadakane on ranks time: 00:00:00
  Sanity-checking and returning
Building samples
Reserving space for 12 sample suffixes
Generating random suffixes
QSorting 12 sample offsets, eliminating duplicates
QSorting sample offsets, eliminating duplicates time: 00:00:00
Multikey QSorting 12 samples
  (Usi

Renaming /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000006765.1/index/GCF_000006765.1_index.3.bt2.tmp to /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000006765.1/index/GCF_000006765.1_index.3.bt2
Renaming /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000006765.1/index/GCF_000006765.1_index.4.bt2.tmp to /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000006765.1/index/GCF_000006765.1_index.4.bt2
Renaming /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000006765.1/index/GCF_000006765.1_index.1.bt2.tmp to /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000006765.1/index/GCF_000006765.1_index.1.bt2
Renaming /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000006765.1/index/GCF_000006765.1_index.2.bt2.tmp to /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000006765.1/index/GCF_00000676

GCF_000014625.1.dict already exists.
Settings:
  Output files: "/media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000014625.1/index/GCF_000014625.1_index.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000014625.1/N_GCF_000014625.1.fasta
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according t

Renaming /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000014625.1/index/GCF_000014625.1_index.3.bt2.tmp to /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000014625.1/index/GCF_000014625.1_index.3.bt2
Renaming /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000014625.1/index/GCF_000014625.1_index.4.bt2.tmp to /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000014625.1/index/GCF_000014625.1_index.4.bt2
Renaming /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000014625.1/index/GCF_000014625.1_index.1.bt2.tmp to /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000014625.1/index/GCF_000014625.1_index.1.bt2
Renaming /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000014625.1/index/GCF_000014625.1_index.2.bt2.tmp to /media/student/EXTERNAL_USB/Variant_Calling/Analysis/genomes/indexed/GCF_000014625.1/index/GCF_00001462

## Bowtie2 alignment

In [8]:
if configuration["bowtie2"]["enabled"]:
    for organism in configuration["bowtie2"]["organisms"]:
        genome = organism["accession"]
        samples = organism["samples"]
            
        print(f"{len(samples)} samples will be aligned with {genome} if possible...")

        index_assembly_directory_path = os.path.join(genomes_directory_path, "indexed", genome, "index", f"{genome}_index")

        aligned_count = 0

        print()
    
        for sample in samples:
            aligned_count += 1        
            start_time = time.time()

            # Decompress R1 and R2 files
            compressed_R1_file_path = os.path.join(fastq_directory_path, f"{sample}_R1.fastq.gz")
            compressed_R2_file_path = os.path.join(fastq_directory_path, f"{sample}_R2.fastq.gz")

            decompressed_R1_file_path = os.path.join(fastq_directory_path, f"{sample}_R1.fastq")
            decompressed_R2_file_path = os.path.join(fastq_directory_path, f"{sample}_R2.fastq")
            
            with gzip.open(compressed_R1_file_path, 'rb') as compressed_R1_file:
                with open(decompressed_R1_file_path, 'wb') as decompressed_R1_file:
                    shutil.copyfileobj(compressed_R1_file, decompressed_R1_file)

            with gzip.open(compressed_R2_file_path, 'rb') as compressed_R2_file:
                with open(decompressed_R2_file_path, 'wb') as decompressed_R2_file:
                    shutil.copyfileobj(compressed_R2_file, decompressed_R2_file)
            
            reference_genome_index = index_assembly_directory_path
            
            # Bowtie2 alignment
            print(f"Aligning {sample}...")
            
            R1_file = decompressed_R1_file_path
            R2_file = decompressed_R2_file_path

            samfiles_file_path = os.path.join(samfiles_directory_path, f"{sample}_map{genome}.sam")
    
            command = [
                bowtie2, "--phred33", "-x", reference_genome_index,
                "-q", "-1", R1_file, "-2", R2_file,
                "-X", "1000", "-S", samfiles_file_path
            ]

            subprocess.run(command, check=True)
            
            end_time = time.time()
            execution_time = end_time - start_time
            
            print()
            print(f"{sample} aligned. Execution time {execution_time:.2f} s")
            print()
            print(f"Aligned {aligned_count}/{len(samples)} with {genome}")
            
            if configuration["compress"]:

                print()
                print(f"Compressing .sam {sample} file")
                
                # Compress resulting .sam file and remove .sam file
                compressed_sam_file_path = os.path.join(samfiles_directory_path, f"{samfiles_file_path}.gz")

                with open(samfiles_file_path, 'rb') as sam_file:
                    with gzip.open(compressed_sam_file_path, 'wb') as compressed_sam_file:
                        shutil.copyfileobj(sam_file, compressed_sam_file)

                os.remove(samfiles_file_path)
            
            # Remove decompressed .fastq files
            os.remove(decompressed_R1_file_path)
            os.remove(decompressed_R2_file_path)

        
        print()

1 samples will be aligned with GCF_000006765.1 if possible...

Aligning PAO1wt12P...


3281626 reads; of these:
  3281626 (100.00%) were paired; of these:
    73435 (2.24%) aligned concordantly 0 times
    3151065 (96.02%) aligned concordantly exactly 1 time
    57126 (1.74%) aligned concordantly >1 times
    ----
    73435 pairs aligned concordantly 0 times; of these:
      12472 (16.98%) aligned discordantly 1 time
    ----
    60963 pairs aligned 0 times concordantly or discordantly; of these:
      121926 mates make up the pairs; of these:
        80327 (65.88%) aligned 0 times
        40207 (32.98%) aligned exactly 1 time
        1392 (1.14%) aligned >1 times
98.78% overall alignment rate



PAO1wt12P aligned. Execution time 1980.75 s

Aligned 1/1 with GCF_000006765.1

2 samples will be aligned with GCF_000014625.1 if possible...

Aligning PA14wt0253G...


6236138 reads; of these:
  6236138 (100.00%) were paired; of these:
    122207 (1.96%) aligned concordantly 0 times
    5984387 (95.96%) aligned concordantly exactly 1 time
    129544 (2.08%) aligned concordantly >1 times
    ----
    122207 pairs aligned concordantly 0 times; of these:
      39330 (32.18%) aligned discordantly 1 time
    ----
    82877 pairs aligned 0 times concordantly or discordantly; of these:
      165754 mates make up the pairs; of these:
        107625 (64.93%) aligned 0 times
        55538 (33.51%) aligned exactly 1 time
        2591 (1.56%) aligned >1 times
99.14% overall alignment rate



PA14wt0253G aligned. Execution time 3687.32 s

Aligned 1/2 with GCF_000014625.1
Aligning PA14wt0251G...

PA14wt0251G aligned. Execution time 1734.67 s

Aligned 2/2 with GCF_000014625.1



2717352 reads; of these:
  2717352 (100.00%) were paired; of these:
    39751 (1.46%) aligned concordantly 0 times
    2625130 (96.61%) aligned concordantly exactly 1 time
    52471 (1.93%) aligned concordantly >1 times
    ----
    39751 pairs aligned concordantly 0 times; of these:
      9881 (24.86%) aligned discordantly 1 time
    ----
    29870 pairs aligned 0 times concordantly or discordantly; of these:
      59740 mates make up the pairs; of these:
        35788 (59.91%) aligned 0 times
        23034 (38.56%) aligned exactly 1 time
        918 (1.54%) aligned >1 times
99.34% overall alignment rate


## SAMtools realign NCBI RefSeq assembly

In [9]:
if configuration["realign"]["enabled"]:
    
    sam_files = os.listdir(samfiles_directory_path)

    for sam_file in sam_files:
        sample = sam_file.split("_")[0]
        
        print(f"Processing {sample}...")
        print()
    
        sam_file_path = os.path.join(samfiles_directory_path, sam_file)

        if sam_file.endswith(".sam.gz"):
            bam_file = sam_file.removesuffix(".sam.gz") + ".bam"
            #bam_file = sam_file[:-7]+"bam"
        else:
            bam_file = sam_file.removesuffix(".sam") + ".bam"
            #bam_file = sam_file[:-4]+"bam"
    
        bam_file_path = os.path.join(intermediate_directory_path, bam_file)

        # Calling samtools view
        command = [
            samtools, "view", "-b", "-S", sam_file_path
        ]
    
        with open(bam_file_path, 'wb') as bam_out_file:
            subprocess.run(command, stdout=bam_out_file, check=True)
    
        # Sort
        sorted_bam_file_path = bam_file_path.removesuffix(".bam") + "_sorted.bam"
        #sorted_bam_file_path = bam_file_path[:-4] + "_sorted.bam"

        command = [
            "java", "-jar", picard_jar, "SortSam",
            "INPUT=" + bam_file_path, "OUTPUT=" + sorted_bam_file_path,
            "SORT_ORDER=coordinate"
        ]

        subprocess.run(command, check=True)
    
        # Add readgroup information
        addrg_bam_file_path = sorted_bam_file_path.removesuffix(".bam") + "_addrg.bam"
        #addrg_bam_file_path = bam_file_path[:-4] + "_addrg.bam"

        read_group_library = f"{main_directory_path}/{sample}_WGS"
        read_group_platform = "Illumina"
        read_group_platform_unit = f"{main_directory_path}/{sample}_WGS"
        read_group_sample_name = f"{main_directory_path}/{sample}_WGS"

        command = [
            "java", "-jar", picard_jar, "AddOrReplaceReadGroups",
            "INPUT=" + sorted_bam_file_path,
            "OUTPUT=" + addrg_bam_file_path,
            "LB=" + read_group_library,
            "PL=" + read_group_platform,
            "PU=" + read_group_platform_unit,
            "SM=" + read_group_sample_name
        ]

        subprocess.run(command, check=True)
    
        # Mark duplicates
        dedup_bam_file_path = addrg_bam_file_path.removesuffix(".bam") + "_dedup.bam"
        #dedup_sorted_bam_file_path = sorted_bam_file_path[:-4] + "_dedup.bam"
        metrics_file_path = bam_file_path.removesuffix(".bam") + "_metrics.txt"
        #metrics_file_path = bam_file_path[:-4] + "_metrics.txt"

        command = [
            "java", "-jar", picard_jar, "MarkDuplicates",
            "INPUT=" + addrg_bam_file_path, "OUTPUT=" + dedup_bam_file_path,
            "METRICS_FILE=" + metrics_file_path
        ]

        subprocess.run(command, check=True)
    
        #Index .bam
        bai_file_path = dedup_bam_file_path.removesuffix("bam") + "bai"
        #bai_file_path = bam_file_path[:-3] + "bai"

        command = [
            "java", "-jar", picard_jar, "BuildBamIndex",
            "INPUT=" + dedup_bam_file_path,
            "OUTPUT=" + bai_file_path
        ]

        subprocess.run(command, check=True)
    
        #Calling GATK
        reference_genome = realign_sample_genome_dict[sample]
    
        reference_genome_fasta_file = f"N_{reference_genome}.fasta"
        reference_genome_fasta_file_path = os.path.join(indexed_directory_path, reference_genome, reference_genome_fasta_file)

        realigned_intervals_file_path = bam_file_path.removesuffix(".bam") + ".realigned.intervals"
        #realigned_intervals_file_path = bam_file_path[:-4] + ".realigned.intervals"

        command = [
            java_8, "-jar", GATK_jar, "-T", "RealignerTargetCreator",
            "-I", dedup_bam_file_path, "-R", reference_genome_fasta_file_path,
            "-o", realigned_intervals_file_path
        ]

        subprocess.run(command, check=True)

        realigned_bam_file_path = bam_file_path.removesuffix(".bam") + ".realigned.bam"
        #realigned_bam_file_path = bam_file_path[:-4] + ".realigned.bam"

        command = [
            java_8, "-jar", GATK_jar, "-T", "IndelRealigner",
            "-I", dedup_bam_file_path, "-R", reference_genome_fasta_file_path,
            "--maxConsensuses", "60", "--maxReadsForConsensuses", "240","--maxReadsForRealignment", "6000",
            "--targetIntervals", realigned_intervals_file_path,
            "-o", realigned_bam_file_path
        ]

        subprocess.run(command, check=True)
    
        # Calling samtools sort
        realigned_sorted_file_path = realigned_bam_file_path.removesuffix(".bam") + ".sorted"
        #realigned_sorted_file_path = realigned_bam_file_path[:-4] + ".sorted"

        command = [
            samtools, "sort", realigned_bam_file_path,
            realigned_sorted_file_path
        ]

        subprocess.run(command, check=True)
    
        # totalpileup files are used for indel calling
        realigned_sorted_bam_file_path = realigned_sorted_file_path + ".bam"
        realigned_totalpileup_file = bam_file.removesuffix(".bam") + ".realigned.totalpileup"
        #realigned_totalpileup_file = bam_file[:-4] + ".realigned.totalpileup"
        realigned_totalpileup_file_path = os.path.join(totalpileup_directory_path, realigned_totalpileup_file)

        with open(realigned_totalpileup_file_path, 'w') as totalpileup_file:
            command = [
                samtools, "pileup", "-c", "-f", reference_genome_fasta_file_path,
                realigned_sorted_bam_file_path
            ]

            subprocess.run(command, stdout=totalpileup_file, check=True)
    
        # raw files are used for SNP calling
        realigned_raw_file = bam_file.removesuffix(".bam") + ".realigned.raw"
        #realigned_raw_file = bam_file[:-] + ".realigned.raw"
        realigned_raw_file_path = os.path.join(raw_directory_path, realigned_raw_file)

        with open(realigned_raw_file_path, 'w') as raw_file:
            command = [
                samtools, "pileup", "-vc", "-f", reference_genome_fasta_file_path,
                realigned_sorted_bam_file_path
            ]

            subprocess.run(command, stdout=raw_file, check=True)

        print()
        print(f"{sample} processed!!")
        print()

    shutil.rmtree(intermediate_directory_path)

Processing PA14wt0251G...



[samopen] SAM header is present: 1 sequences.
INFO	2025-06-04 11:10:14	SortSam	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** 
https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    SortSam -INPUT /media/student/EXTERNAL_USB/Variant_Calling/Analysis/variant_calling/intermediate/PA14wt0251G_mapGCF_000014625.1.bam -OUTPUT /media/student/EXTERNAL_USB/Variant_Calling/Analysis/variant_calling/intermediate/PA14wt0251G_mapGCF_000014625.1_sorted.bam -SORT_ORDER coordinate
**********


11:10:14.800 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/student/miniconda3/envs/VariantCalling/share/picard-3.4.0-0/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Jun 04 11:10:14 CEST 2025] SortSam INPUT=/media/student/EXTERNAL_USB/Variant_C


PA14wt0251G processed!!

Processing PA14wt0253G...



[samopen] SAM header is present: 1 sequences.
INFO	2025-06-04 11:39:53	SortSam	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** 
https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    SortSam -INPUT /media/student/EXTERNAL_USB/Variant_Calling/Analysis/variant_calling/intermediate/PA14wt0253G_mapGCF_000014625.1.bam -OUTPUT /media/student/EXTERNAL_USB/Variant_Calling/Analysis/variant_calling/intermediate/PA14wt0253G_mapGCF_000014625.1_sorted.bam -SORT_ORDER coordinate
**********


11:39:53.301 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/student/miniconda3/envs/VariantCalling/share/picard-3.4.0-0/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Jun 04 11:39:53 CEST 2025] SortSam INPUT=/media/student/EXTERNAL_USB/Variant_C


PA14wt0253G processed!!

Processing PAO1wt12P...



[samopen] SAM header is present: 1 sequences.
INFO	2025-06-04 12:41:32	SortSam	

********** NOTE: Picard's command line syntax is changing.
**********
********** For more information, please see:
********** 
https://github.com/broadinstitute/picard/wiki/Command-Line-Syntax-Transition-For-Users-(Pre-Transition)
**********
********** The command line looks like this in the new syntax:
**********
**********    SortSam -INPUT /media/student/EXTERNAL_USB/Variant_Calling/Analysis/variant_calling/intermediate/PAO1wt12P_mapGCF_000006765.1.bam -OUTPUT /media/student/EXTERNAL_USB/Variant_Calling/Analysis/variant_calling/intermediate/PAO1wt12P_mapGCF_000006765.1_sorted.bam -SORT_ORDER coordinate
**********


12:41:32.459 INFO  NativeLibraryLoader - Loading libgkl_compression.so from jar:file:/home/student/miniconda3/envs/VariantCalling/share/picard-3.4.0-0/picard.jar!/com/intel/gkl/native/libgkl_compression.so
[Wed Jun 04 12:41:32 CEST 2025] SortSam INPUT=/media/student/EXTERNAL_USB/Variant_Calli


PAO1wt12P processed!!



## Find indels without common

In [10]:
if configuration["indels"]:

    totalpileup_files = os.listdir(totalpileup_directory_path)

    for file in totalpileup_files:
        totalpileup_file_path = os.path.join(totalpileup_directory_path, file)
    
        indels_withoutcommon_file = file.removesuffix(".realigned.totalpileup") + ".withoutcommon.indels"
        indels_withoutcommon_file_path = os.path.join(indels_directory_path, indels_withoutcommon_file)

        with open(indels_withoutcommon_file_path, 'w') as indels_wc_file:
            with open(totalpileup_file_path, 'r') as totalpileup_file:
        
                indels = []
        
                for line in totalpileup_file:
                    fields = line.strip().split()

                    if fields[2] == "*":
                        if int(fields[5]) >= 500 and int(fields[6]) >= 25:
                            if (
                                (fields[8] == "*" and int(fields[11])*5 > int(fields[7])) or
                                (fields[9] == "*" and int(fields[10])*5 >= int(fields[7])) or
                                (fields[8] != "*" and fields[9] != "*" and (int(fields[11])*5 >= int(fields[7]) or
                                 int(fields[10])*5 >= int(fields[7])))
                            ):
                                indels.append(line)

            indels_wc_file.writelines(indels)

## Find SNPs without common

In [11]:
if configuration["snps"]:
    raw_files = os.listdir(raw_directory_path)

    for file in raw_files:
        raw_file_path = os.path.join(raw_directory_path, file)
    
        snps_withoutcommon_file = file.removesuffix(".realigned.raw") + ".snps.withoutcommon"
        snps_withoutcommon_file_path = os.path.join(snps_directory_path, snps_withoutcommon_file)

        with open(snps_withoutcommon_file_path, 'w') as snps_wc_file:
            with open(raw_file_path, 'r') as raw_file:
        
                snps = []
        
                for line in raw_file:
                    fields = line.strip().split()

                    if (int(fields[5]) >= 50 and int(fields[6]) >= 25 and int(fields[7]) >= 3):
                        if fields[3] not in ["M","R","W","S","Y","K"]:
                            snps.append(line)

            snps_wc_file.writelines(snps)

## Convert .withoutcommon files to .vcf

In [12]:
if configuration["SNP_indel_annotation"]["enabled"]:
    snps_files = os.listdir(snps_directory_path)

    for file in snps_files:
        snps_file_path = os.path.join(snps_directory_path, file)
        vcf_file = file.removesuffix(".raw") + ".vcf"
        vcf_file_path = os.path.join(vcf_directory_path, vcf_file)

        with open(vcf_file_path, 'w') as vcf_out_file:
            command = [
                perl, convert2annovar, snps_file_path
            ]

            subprocess.run(command, stdout=vcf_out_file, check=True)

NOTICE: the default --format argument is set as 'pileup'
NOTICE: the default --snpqual argument for pileup format is set as 20
NOTICE: Column 6-9 in output are heterozygosity status, SNP quality, total reads, reads with mutation
NOTICE: Read 271 lines and wrote 275 different variants at 271 genomic positions (127 SNPs and 144 indels)
NOTICE: Among 275 different variants at 271 positions, 71 are heterozygotes, 204 are homozygotes
NOTICE: Among 127 SNPs, 80 are transitions, 47 are transversions
NOTICE: the default --format argument is set as 'pileup'
NOTICE: the default --snpqual argument for pileup format is set as 20
NOTICE: Column 6-9 in output are heterozygosity status, SNP quality, total reads, reads with mutation
NOTICE: Read 307 lines and wrote 309 different variants at 307 genomic positions (126 SNPs and 181 indels)
NOTICE: Among 309 different variants at 307 positions, 101 are heterozygotes, 208 are homozygotes
NOTICE: Among 126 SNPs, 78 are transitions, 48 are transversions
NOT

## Build genome database

In [13]:
# https://pcingola.github.io/SnpEff/snpeff/build_db/#step-2-option-2-building-a-database-from-genbank-files
# https://pcingola.github.io/SnpEff/snpeff/faq/#how-to-building-an-ncbi-genome-genbank-file
if configuration["build"]["enabled"]:
    for database in configuration["build"]["databases"]:
        command = [
            buildDbNcbi, database
        ]

        subprocess.run(command, cwd = snpeff_dir_path, check=True)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

100 14.0M    0 14.0M    0     0   904k      0 --:--:--  0:00:15 --:--:-- 1834k
00:00:00 SnpEff version SnpEff 5.2 (build 2023-09-29 06:17), by Pablo Cingolani
00:00:00 Command: 'build'
00:00:00 Building database for 'NC_002516.2'
00:00:00 Reading configuration file 'snpEff.config'. Genome: 'NC_002516.2'
00:00:00 Reading config file: /home/student/miniconda3/envs/VariantCalling/share/snpeff-5.2-1/snpEff.config
00:00:01 done
00:00:04 Chromosome: 'NC_002516.2'	length: 6264404
00:00:04 Create exons from CDS (if needed): 


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

00:00:04 Exons created for 5572 transcripts.
00:00:04 Deleting redundant exons (if needed): 
00:00:04 	Total transcripts with deleted exons: 0
00:00:04 Collapsing zero length introns (if needed): 
00:00:04 	Total collapsed transcripts: 0
00:00:04 		Adding genomic sequences to genes: 
00:00:05 	Done (4779 sequences added).
00:00:05 		Adding genomic sequences to exons: 
00:00:05 	Done (5573 sequences added, 0 ignored).
00:00:05 Finishing up genome
00:00:05 Adjusting transcripts: 
00:00:05 Adjusting genes: 
00:00:05 Adjusting chromosomes lengths: 
00:00:05 Ranking exons: 
00:00:05 Create UTRs from CDS (if needed): 
00:00:05 Remove empty chromosomes: 
00:00:05 Marking as 'coding' from CDS information: 
00:00:05 Done: 0 transcripts marked
00:00:05 
00:00:05 Caracterizing exons by splicing (stage 1) : 
.....
00:00:05 Caracterizing exons by splicing (stage 2) : 
.....00:00:05 done.
00:00:05 [Optional] Rare amino acid annotations
WARNING_FILE_NOT_FOUND: Rare Amino Acid analysis: Cannot read pr

			+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
	++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
	++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
	++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
	++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
	++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
	++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
	++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
	++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
	++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

...

100 15.7M    0 15.7M    0     0   785k      0 --:--:--  0:00:20 --:--:-- 1222k
00:00:00 SnpEff version SnpEff 5.2 (build 2023-09-29 06:17), by Pablo Cingolani
00:00:00 Command: 'build'
00:00:00 Building database for 'NC_008463.1'
00:00:00 Reading configuration file 'snpEff.config'. Genome: 'NC_008463.1'
00:00:00 Reading config file: /home/student/miniconda3/envs/VariantCalling/share/snpeff-5.2-1/snpEff.config
00:00:01 done
00:00:04 Chromosome: 'NC_008463.1'	length: 6537648
00:00:04 Create exons from CDS (if needed): 


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

00:00:04 Exons created for 5961 transcripts.
00:00:04 Deleting redundant exons (if needed): 
00:00:04 	Total transcripts with deleted exons: 0
00:00:04 Collapsing zero length introns (if needed): 
00:00:04 	Total collapsed transcripts: 0
00:00:04 		Adding genomic sequences to genes: 
00:00:05 	Done (4989 sequences added).
00:00:05 		Adding genomic sequences to exons: 
00:00:05 	Done (5966 sequences added, 0 ignored).
00:00:05 Finishing up genome
00:00:05 Adjusting transcripts: 
00:00:05 Adjusting genes: 
00:00:05 Adjusting chromosomes lengths: 
00:00:05 Ranking exons: 
00:00:05 Create UTRs from CDS (if needed): 
00:00:05 Remove empty chromosomes: 
00:00:05 Marking as 'coding' from CDS information: 
00:00:05 Done: 70 transcripts marked
00:00:05 
00:00:05 Caracterizing exons by splicing (stage 1) : 
.....
00:00:05 Caracterizing exons by splicing (stage 2) : 
.....00:00:05 done.
00:00:05 [Optional] Rare amino acid annotations
WARNING_FILE_NOT_FOUND: Rare Amino Acid analysis: Cannot read p

			+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
	++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
	++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++.+++++++++++++++++++++++++++
	++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
	+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++.++++++++++++
	+.++++++++++++++++++++++++++++++++++++++++++++++++++++++++.++++++++++.++++++++++++++++++++++++++++++
	+++++++++++++.++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
	++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
	++++++++++++++++++++.++++++++++++++++++++++++++++++++++++++++++++++++.++++++++++++++++++++++++++++++
	++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

00:00:07 

00:00:07 Saving database
00:00:09 Saving sequences for chromosome 'NC_008463.1' to file '/home/student/miniconda3/envs/VariantCalling/share/snpeff-5.2-1/./data/NC_008463.1/sequence.NC_008463.1.bin'
00:00:10 [Optional] Reading regulation elements: GFF
WARNING_FILE_NOT_FOUND: Cannot read optional regulation file '/home/student/miniconda3/envs/VariantCalling/share/snpeff-5.2-1/./data/NC_008463.1/regulation.gff', nothing done.
00:00:10 [Optional] Reading regulation elements: BED 
00:00:10 Cannot find optional regulation dir '/home/student/miniconda3/envs/VariantCalling/share/snpeff-5.2-1/./data/NC_008463.1/regulation.bed/', nothing done.
00:00:10 [Optional] Reading motifs: GFF
WARNING_FILE_NOT_FOUND: Cannot open PWMs file /home/student/miniconda3/envs/VariantCalling/share/snpeff-5.2-1/./data/NC_008463.1/pwms.bin. Nothing done
00:00:10 Done
00:00:10 Logging
00:00:11 Checking for updates...
00:00:13 Done.


## SNPs and indels annotation

In [14]:
if configuration["SNP_indel_annotation"]["enabled"]:
    vcf_files = [file for file in os.listdir(vcf_directory_path) if file.endswith(".snps.withoutcommon.vcf")]

    for file in vcf_files:
        vcf_file_path = os.path.join(vcf_directory_path, file)
    
        vcf4snpEff_file_path = vcf_file_path + "4snpEff"

        with open(vcf4snpEff_file_path, 'w') as vcf4snpEff_file:
            with open(vcf_file_path, 'r') as vcf_file:
                lines = []

                for line in vcf_file:
                    fields = line.strip().split()

                    if fields[3] != "-" and fields[4] != "-":
                        lines.append(line)

            vcf4snpEff_file.writelines(lines)

    vcf4snpEff_files = [file for file in os.listdir(vcf_directory_path) if file.endswith("vcf4snpEff")]

    for file in vcf4snpEff_files:
        sample = file.strip().split("_")[0]
        genome = annotation_sample_genome_dict[sample]
    
        snps_wc_annotated_file = file.removesuffix(".vcf4snpEff") + ".annotated"
        snps_wc_annotated_file_path = os.path.join(annotated_directory_path, snps_wc_annotated_file)

        vcf4snpEff_file_path = os.path.join(vcf_directory_path, file)

        with open(snps_wc_annotated_file_path, 'w') as snps_annotated_file:
    
            command = [
                java_21, "-jar", snpEff_jar,
                "-v", genome, vcf4snpEff_file_path
            ]

            subprocess.run(command, stdout=snps_annotated_file, cwd=snpeff_dir_path, check=True)
        snps_annotated_file.close()
    
        temp_directory_path = os.path.join(variant_calling_directory_path, "temp")
    
        if not os.path.exists(temp_directory_path):
            os.mkdir(temp_directory_path)

        temp1_file_path = os.path.join(temp_directory_path, "temp1")
    
        with open(temp1_file_path, 'w') as temp1_file:
            with open(snps_wc_annotated_file_path, 'r') as snps_annotated_file:
                command = [
                    perl, vcfEffOnePerLine
                ]

                subprocess.run(command, stdin=snps_annotated_file, stdout=temp1_file, check=True)

        temp2_file_path = os.path.join(temp_directory_path, "temp2")
    
        with open(temp2_file_path, 'w') as temp2_file:

            command = [
                java_21, "-jar", snpSift_jar, "extractFields",
                temp1_file_path, "POS", "REF", "ALT", "ANN[*].EFFECT",
                "ANN[*].IMPACT", "ANN[*].GENE", "ANN[*].GENEID",
                "ANN[*].HGVS_C", "ANN[*].HGVS_P", "ANN[*].CDNA_POS",
                "ANN[*].CDNA_LEN", "ANN[*].AA_POS", "ANN[*].AA_LEN"
            ]

            subprocess.run(command, stdout=temp2_file, check=True)
        temp2_file.close()

        semicurated_file = file.removesuffix(".vcf4snpEff") + ".semicurated"
        semicurated_file_path = os.path.join(curated_directory_path, semicurated_file)
    
        with open(semicurated_file_path, 'w') as semicurated_file:
            with open(temp2_file_path, 'r') as temp2_file:

                content = temp2_file.readlines()
            semicurated_file.writelines(content)
        
        curated_file = file.removesuffix(".vcf4snpEff") + ".curated"
        curated_file_path = os.path.join(curated_directory_path, curated_file)
    
        with open(curated_file_path, 'w') as curated_file:
            with open(semicurated_file_path, 'r') as semicurated_file:
            
                curated_lines = []
            
                for line in semicurated_file:
                    fields = line.strip().split()

                    if fields[4] == "MODERATE" or fields[4] == "HIGH":
                        curated_lines.append(line)

            curated_file.writelines(curated_lines)

        shutil.rmtree(temp_directory_path)

00:00:00 SnpEff version SnpEff 5.2 (build 2023-09-29 06:17), by Pablo Cingolani
00:00:00 Command: 'ann'
00:00:00 Reading configuration file 'snpEff.config'. Genome: 'NC_008463.1'
00:00:00 Reading config file: /home/student/miniconda3/envs/VariantCalling/share/snpeff-5.2-1/snpEff.config
00:00:01 done
00:00:01 Reading database for genome version 'NC_008463.1' from file '/home/student/miniconda3/envs/VariantCalling/share/snpeff-5.2-1/./data/NC_008463.1/snpEffectPredictor.bin' (this might take a while)
00:00:03 done
00:00:03 Loading Motifs and PWMs
00:00:03 Building interval forest
00:00:03 done.
00:00:03 Genome stats :
#-----------------------------------------------
# Genome name                : 'NC_008463.1'
# Genome version             : 'NC_008463.1'
# Genome ID                  : 'NC_008463.1[0]'
# Has protein coding info    : true
# Has Tr. Support Level info : true
# Genes                      : 6041
# Protein coding genes       : 5961
#--------------------------------------------

## Filter by parental

In [15]:
if configuration["filter_by_parental"]["enabled"]:
    
    # Filter SNPs
    curated_files = [file for file in os.listdir(curated_directory_path) if file.endswith(".curated")]
    
    for file in curated_files:    
        sample = file.split("_")[0]
        
        curated_file_path = os.path.join(curated_directory_path, file)
        filtered_file_path = os.path.join(curated_directory_path, file + ".filtered")
            
        with open(curated_file_path, 'r') as curated_file:
            snp_lines = curated_file.readlines()

        processed_lines = []
        for line in snp_lines:
            fields = line.strip().split("\t")
            if len(fields) >= 9:
                processed_lines.append("\t".join([fields[0], fields[5], fields[8]]))

        processed_lines.sort()

        for parental in configuration["filter_by_parental"]["Parental"]:
            if sample in parental["samples"]:
                snp_path = parental["snps_path"]

        with open(snp_path, 'r') as parental_file:
            parental_lines = []
            for line in parental_file:
                fields = line.strip().split("\t")
                if len(fields) >= 9:
                    processed_line = "\t".join([fields[0], fields[5], fields[8]])
                    parental_lines.append(processed_line)

        filtered_lines = []
        for line in snp_lines:
            fields = line.strip().split("\t")
            compare = "\t".join([fields[0], fields[5], fields[8]])
            if compare not in parental_lines:
                filtered_lines.append(line)

        with open(filtered_file_path, 'w') as filtered_file:
            for line in filtered_lines:
                filtered_file.write(line + "\n")

    # Filter indels
    indels_files = [file for file in os.listdir(indels_directory_path) if file.endswith(".indels")]
        
    for file in indels_files:
        sample = file.strip().split("_")[0]

        indels_file_path = os.path.join(indels_directory_path, file)
        filtered_file_path = os.path.join(indels_directory_path, file + ".filtered")
        
        with open(indels_file_path, 'r') as indels_file:
            sample_indels = indels_file.readlines()

        for parental in configuration["filter_by_parental"]["Parental"]:
            if sample in parental["samples"]:
                ind_file_path = parental["indels_path"]

        with open(ind_file_path, 'r') as parental_file:
            parental_lines = parental_file.readlines()

        processed_lines = []
        for line in parental_lines:
            fields = line.strip().split("\t")
            processed_lines.append("\t".join(fields[1:4]))

        filtered_lines = []

        for line in sample_indels:
            fields = line.strip().split("\t")
            compare = "\t".join(fields[1:4])
            if compare not in processed_lines:
                filtered_lines.append(line)

        with open(filtered_file_path, 'w') as filtered_file:
            for line in filtered_lines:
                filtered_file.write(line + "\n")
    

## Create mutations table

In [16]:
if configuration["table"]["enabled"]:
    
    filtered_files = [file for file in os.listdir(curated_directory_path) if file.endswith(".filtered")]
    if len(filtered_files) > 0:
        snp_files = filtered_files
    else:
        snp_files = [file for file in os.listdir(curated_directory_path) if file.endswith(".curated")]
    
    # Add sample column
    for file in snp_files:
        snps_file_path = os.path.join(curated_directory_path, file)
        try:
            df = pd.read_csv(snps_file_path, sep="\t", header=0)
            df[file] = file
            df.to_csv(snps_file_path, index=False, sep="\t")
        except pd.errors.EmptyDataError:
            sample = file.strip().split("_")[0]
            print(f"The sample {sample} does not contain any SNPs.")

    filtered_files = [file for file in os.listdir(indels_directory_path) if file.endswith(".filtered")]
    if len(filtered_files) > 0:
        ind_files = filtered_files
    else:
        ind_files = [file for file in os.listdir(indels_directory_path) if file.endswith(".indels")]
    
    for file in ind_files:
        ind_file_path = os.path.join(indels_directory_path, file)
        try:
            df = pd.read_csv(ind_file_path, sep="\t", header=0)
            df[file] = file
            df.to_csv(ind_file_path, index=False, sep="\t")
        except EmptyDataError:
            sample = file.strip().split("_")[0]
            print(f"The sample {sample} does not contain any indel.") 

    # SNP identification
    information_directory_path = os.path.join(curated_directory_path, "information")

    if not os.path.exists(information_directory_path):
        os.mkdir(information_directory_path)

    ## Obtain old_locus_tag for Pseudomonas aeruginosa UCBPP-PA14 ##
    # ------------------------------------------------------------ #
    
    gff3 = os.path.join(annotated_genomes_directory_path, "PA14.gff3")
    locus_file_path = os.path.join(information_directory_path, "PA14_locus_tags.txt")

    PA14_locus_dict = {}

    with open(gff3, 'r') as gff3_file:
        for line in gff3_file:
            if line.startswith("#"):
                continue
        
            fields = line.strip().split("\t")
            if len(fields) < 9:
                continue
            
            if fields[1] == "RefSeq" and fields[2] == "gene":
                attributes = fields[8]
                attributes_dict = {}

                for attribute in attributes.split(";"):
                    if "=" in attribute:
                        key, value = attribute.split("=", 1)
                        attributes_dict[key] = value
                    if "locus_tag" in attributes_dict:
                        locus = attributes_dict["locus_tag"]
                        old_locus = attributes_dict.get("old_locus_tag", locus)
                        PA14_locus_dict[locus] = old_locus

    with open(locus_file_path, 'w') as locus_file:
        for locus_tag, old_locus_tag in PA14_locus_dict.items():
            locus_file.write(f"{locus_tag}\t{old_locus_tag}\n")

    # -------------------------------------------------------------------------- #
    ##############################################################################

    ## Filter SNPs
    snps_dict = {}

    for file in snp_files:
        file_path = os.path.join(curated_directory_path, file)
        sample = file.strip().split("_")[0]

        with open(file_path, 'r') as snp_file:
            snp_file_reader = csv.reader(snp_file, delimiter="\t")

            for row in snp_file_reader:
                gene = row[5]
                locus = row[6]
                aa = row[8]

                #########################################
                # Modified for PA14 strain locus change #
                # ------------------------------------- #
                # snp = ",".join([locus, gene, aa])     #
                #########################################
                
                if table_sample_strain_dict[sample] == "PA14":
                    old_locus = PA14_locus_dict[locus]
                    snp = ",".join([old_locus, gene, aa])
                else:
                    snp = ",".join([locus, gene, aa])

                if snp not in snps_dict:
                    snps_dict[snp] = 1
                else:
                    snps_dict[snp] += 1

    snps_info_file_path = os.path.join(information_directory_path, "snps_info.txt")
    
    with open(snps_info_file_path, 'w') as snps_info_file:

        for snp, count in snps_dict.items():
            snps_info_file.write(f"{count},{snp}\n")

    filtered_snps_file_path = os.path.join(information_directory_path, "filtered_snps.txt")
    with open(filtered_snps_file_path, 'w') as filtered_snps_file:
        
        for snp, count in snps_dict.items():
            if count <= configuration["table"]["threshold"]:
                filtered_snps_file.write(f"{snp}\n")
    
    ## SNP tabulation
    snp_tab_file_path = os.path.join(information_directory_path, "snp_tabulation.txt")
    with open(snp_tab_file_path, 'w') as snp_tab_file:

        for file in snp_files:
            file_path = os.path.join(curated_directory_path, file)

            with open(file_path, 'r') as snp_file:
                snp_file_reader = csv.reader(snp_file, delimiter="\t")

                for row in snp_file_reader:
                    gene = row[5]
                    locus = row[6]
                    aa = row[8]
                    sample_name = row[13].split("_")[0]

                    #########################################
                    # Modified for PA14 strain locus change #
                    # ------------------------------------- #
                    # snp = ",".join([locus, gene, aa])     #
                    #########################################
                    
                    if table_sample_strain_dict[sample_name] == "PA14":
                        old_locus = PA14_locus_dict[locus]
                        snp = ",".join([old_locus, gene, aa])
                    else:
                        snp = ",".join([locus, gene, aa])

                    if snp in snps_dict and snps_dict[snp] <= configuration["table"]["threshold"]:
                        snp_tab_file.write(",".join([snp, sample_name]) + "\n")

    ## SNPs table
    snp_table_file_path = os.path.join(results_directory_path, "snp_table.csv")
    snps_aa_dict = defaultdict(lambda: defaultdict(list))

    with open(snp_tab_file_path, 'r') as snp_tab_file:
        lines = snp_tab_file.readlines()

        for line in lines:
            row = line.strip("\n").split(",")

            locus = row[0]
            gene = row[1]
            aa = row[2]
            sample = row[3]

            column_name = f"{locus} ({gene})"

            snps_aa_dict[sample][column_name].append(aa)

        with open(snp_table_file_path, 'w', newline='') as snp_table_file:
            snp_table_file_writer = csv.writer(snp_table_file)

            all_columns = set()

            for sample in snps_aa_dict:
                for column_name in snps_aa_dict[sample]:
                    all_columns.add(column_name)

            header = [""] + sorted(list(all_columns))
            snp_table_file_writer.writerow(header)

            for sample, snps in snps_aa_dict.items():
                row = [sample]
                for snp in sorted(list(all_columns)):
                    aa_mutations = ",".join(snps.get(snp, []))
                    row.append(aa_mutations)
                snp_table_file_writer.writerow(row)

    # Indels identification
    information_directory_path = os.path.join(indels_directory_path, "information")
    annotated_ind_directory_path = os.path.join(indels_directory_path, "annotated")

    if not os.path.exists(information_directory_path):
        os.mkdir(information_directory_path)
    if not os.path.exists(annotated_ind_directory_path):
        os.mkdir(annotated_ind_directory_path)

    ## Obtain indels position
    pos_strain_dict = {}

    for file in ind_files:
        file_path = os.path.join(indels_directory_path, file)
        
        sample = file.strip().split("_")[0]
        strain = table_sample_strain_dict[sample]

        if strain not in pos_strain_dict:
            pos_strain_dict[strain] = {
                "Positions": []
            }

        with open(file_path, 'r') as ind_file:
            ind_file_reader = csv.reader(ind_file, delimiter='\t')

            for row in ind_file_reader:
                pos = int(row[1])
                pos_strain_dict[strain]["Positions"].append(pos)

    ## Annotate indels
    annotated_pos_dict = {}

    for file in ind_files:
        sample = file.strip().split("_")[0]
        genome = table_sample_strain_dict[sample]
    
        annotated_genome = table_sample_strain_dict[sample] + ".csv"
        annotated_genome_file_path = os.path.join(annotated_genomes_directory_path, annotated_genome)

        with open(annotated_genome_file_path, 'r') as annotated_genome_file:
            annotated_genome_reader = csv.reader(annotated_genome_file, delimiter=',')

            next(annotated_genome_reader)

            if genome not in annotated_pos_dict:
                annotated_pos_dict[genome] = {}

            for row in annotated_genome_reader:
                locus = row[5]
                start = int(row[7])
                end = int(row[8])
                gene = row[10]

                for position in pos_strain_dict[genome]["Positions"]:
                    if position >= start and position <= end:
                        annotated_pos_dict[genome][position] = {
                            'locus': locus,
                            'gene': gene
                        }
                for position in pos_strain_dict[genome]["Positions"]:
                    if position not in annotated_pos_dict[genome]:
                        annotated_pos_dict[genome][position] = {
                            'locus': 'intron',
                            'gene': 'intron'
                        }

    for file in ind_files:
        file_path = os.path.join(indels_directory_path, file)

        sample = file.strip().split("_")[0]
        genome = table_sample_strain_dict[sample]

        with open(file_path, 'r') as ind_file:
            ind_file_reader = csv.reader(ind_file, delimiter='\t')

            modified_rows = []

            for row in ind_file_reader:
                pos = int(row[1])

                annotation = annotated_pos_dict.get(genome, {}).get(pos, {
                    "refseq": "NA", "locus": "NA", "gene":"NA"
                })

                locus = annotation["locus"]
                gene = annotation["gene"]

                if gene != "":
                    row.extend([locus, gene])
                else:
                    row.extend([locus, locus])
                
                del row[2]

                modified_rows.append(row)

        annotated_ind_file_path = os.path.join(annotated_ind_directory_path, file)

        with open(annotated_ind_file_path, 'w', newline='') as annotated_ind_file:
            annotated_ind_file_writer = csv.writer(annotated_ind_file)
            annotated_ind_file_writer.writerows(modified_rows)


    ## Filter indels
    inds_dict = {}

    for file in ind_files:
        file_path = os.path.join(indels_directory_path, file)

        with open(file_path, 'r') as ind_file:
            ind_file_reader = csv.reader(ind_file, delimiter='\t')

            for row in ind_file_reader:
                refseq = row[0]
                pos = row[1]
                mut = row[3]

                indel = ",".join([refseq, pos, mut])

                if indel not in inds_dict:
                    inds_dict[indel] = 1
                else:
                    inds_dict[indel] += 1

    inds_info_file_path = os.path.join(information_directory_path, "indels_info.txt")
    with open(inds_info_file_path, 'w') as inds_info_file:

        for indel, count in inds_dict.items():
            inds_info_file.write(f"{count}, {indel}\n")

    filtered_inds_file_path = os.path.join(information_directory_path, "filtered_indels.txt")
    with open(filtered_inds_file_path, 'w') as filtered_inds_file:

        for indel, count in inds_dict.items():
            if count <= configuration["table"]["threshold"]:
                filtered_inds_file.write(f"{indel}\n")

    ## Indels tabulation
    ind_tab_file_path = os.path.join(information_directory_path, "indel_tabulation.txt")

    annot_files = [file for file in os.listdir(annotated_ind_directory_path)]
    
    with open(ind_tab_file_path, 'w') as ind_tab_file:
        for file in annot_files:
            file_path = os.path.join(annotated_ind_directory_path, file)

            with open(file_path, 'r') as ind_file:
                ind_file_reader = csv.reader(ind_file, delimiter=",")

                for row in ind_file_reader:
                    refseq = row[0]
                    pos = row[1]
                    mut = row[2]
                    sample = row[14].split("_")[0]
                    locus = row[15]
                    gene = row[16]

                    indel = ",".join([refseq, pos, mut])

                    if indel in inds_dict and inds_dict[indel] <= configuration["table"]["threshold"]:
                        ind_mut = f"{pos} {mut}"
                        ind_tab_file.write(",".join([locus, gene, ind_mut, sample]) + "\n")

    # Indels table
    ind_table_file_path = os.path.join(results_directory_path, "indels_table.csv")
    
    final_inds_dict = defaultdict(lambda: defaultdict(list))

    with open(ind_tab_file_path, 'r') as ind_tab_file:
        lines = ind_tab_file.readlines()

        for line in lines:
            row = line.strip("\n").split(",")
            
            locus = row[0]
            gene = row[1]
            indel = row[2]
            sample = row[3]

            if gene != "":
                column_name = f"{locus} ({gene})"
            else:
                column_name = f"{locus} ({locus})"

            final_inds_dict[sample][column_name].append(indel)

        with open(ind_table_file_path, 'w', newline='') as ind_table_file:
            ind_table_file_writer = csv.writer(ind_table_file)

            all_columns = set()

            for sample in final_inds_dict:
                for column_name in final_inds_dict[sample]:
                    all_columns.add(column_name)

            header = [""] + sorted(list(all_columns))
            ind_table_file_writer.writerow(header)

            for sample, inds in final_inds_dict.items():
                row = [sample]
                for indel in sorted(list(all_columns)):
                    indels = ",".join(inds.get(indel, []))
                    row.append(indels)
                ind_table_file_writer.writerow(row)

    # SNP/indel table
    mutations_file_path = os.path.join(results_directory_path, "mutations.txt")

    with open(mutations_file_path, 'w', newline='') as mutations_file:
        mutations_file_writer = csv.writer(mutations_file)

        with(open(snp_tab_file_path, 'r') as snp_tab_file,
             open(ind_tab_file_path, 'r') as ind_tab_file):

            snp_tab_file_reader = csv.reader(snp_tab_file, delimiter=',')
            ind_tab_file_reader = csv.reader(ind_tab_file, delimiter=',')

            for row in snp_tab_file_reader:
                mutations_file_writer.writerow(row)

            for row in ind_tab_file_reader:
                mutations_file_writer.writerow(row)

    ## mutations table
    combined_table_file_path = os.path.join(results_directory_path, "mutations.csv")

    mut_dict = defaultdict(lambda: defaultdict(list))

    with open(mutations_file_path, 'r') as mut_file:
        lines = mut_file.readlines()

        for line in lines:
            row = line.strip().split(",")

            locus = row[0]
            gene = row[1]
            mut = row[2]
            sample = row[3]

            if gene != "":
                column_name = f"{locus} ({gene})"
            else:
                column_name = f"{locus} ({locus})"

            mut_dict[sample][column_name].append(mut)

    with open(combined_table_file_path, 'w', newline='') as combined_table_file:
        combined_table_file_writer = csv.writer(combined_table_file)

        all_columns = set()

        for sample in mut_dict:
            for column_name in mut_dict[sample]:
                all_columns.add(column_name)

        header = [""] + sorted(list(all_columns))
        combined_table_file_writer.writerow(header)

        for sample, muts in mut_dict.items():
            row = [sample]
            for column in sorted(list(all_columns)):
                mutations = ", ".join(muts.get(column, []))
                row.append(mutations)
            combined_table_file_writer.writerow(row)
            

The sample PA14wt0253G does not contain any SNPs.
The sample PAO1wt12P does not contain any SNPs.


## *De novo* genome assembly

In [17]:
if configuration["denovo"]:

    samples = {file.split("_")[0] for file in os.listdir(fastq_directory_path)}

    for sample in samples:
        files = os.listdir(fastq_directory_path)

        R1_file = sample + "_R1.fastq.gz"
        R1_file_path = os.path.join(fastq_directory_path, R1_file)
        R2_file = sample + "_R2.fastq.gz"
        R2_file_path = os.path.join(fastq_directory_path, R2_file)

        denovo_directory = sample + "_SPAdes"
        denovo_directory_path = os.path.join(assemblies_directory_path, denovo_directory)

        if os.path.exists(denovo_directory_path):
            shutil.rmtree(denovo_directory_path)

        command = [
            "python", spades, "-o", denovo_directory_path,
            "-1", R1_file_path, "-2", R2_file_path,
            "--isolate"
        ]

        subprocess.run(command, check=True)

        contigs_file_path = os.path.join(denovo_directory_path, "contigs.fasta")

        denovo_file = sample + ".SPAdes.denovoassembly.fasta"
        denovo_file_path = os.path.join(assemblies_directory_path, denovo_file)
        
        os.rename(contigs_file_path, denovo_file_path)

        shutil.rmtree(denovo_directory_path)

Command line: /home/student/miniconda3/envs/VariantCalling/bin/spades.py	-o	/media/student/EXTERNAL_USB/Variant_Calling/Analysis/assemblies/PA14wt0253G_SPAdes	-1	/media/student/EXTERNAL_USB/Variant_Calling/Analysis/fastq/PA14wt0253G_R1.fastq.gz	-2	/media/student/EXTERNAL_USB/Variant_Calling/Analysis/fastq/PA14wt0253G_R2.fastq.gz	--isolate	

System information:
  SPAdes version: 4.1.0
  Python version: 3.11.0
  OS: Linux-5.15.0-139-generic-x86_64-with-glibc2.31

Output dir: /media/student/EXTERNAL_USB/Variant_Calling/Analysis/assemblies/PA14wt0253G_SPAdes
Mode: ONLY assembling (without read error correction)
Debug mode is turned OFF

Dataset parameters:
  Isolate mode
  Reads:
    Library number: 1, library type: paired-end
      orientation: fr
      left reads: ['/media/student/EXTERNAL_USB/Variant_Calling/Analysis/fastq/PA14wt0253G_R1.fastq.gz']
      right reads: ['/media/student/EXTERNAL_USB/Variant_Calling/Analysis/fastq/PA14wt0253G_R2.fastq.gz']
      interlaced reads: not specif